libraries

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel


load dataset

In [ ]:
# Load the dataset
df = pd.read_csv("movies_metadata.csv", low_memory=False)

# Display first few rows
print(df.head())


c:\Users\djsce.student\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Preprocessing and filtering


a

In [ ]:
df_new = df.head(10000)
c=df_new['vote_average'].mean()

                                 title  vote_count  vote_average     score
314           The Shawshank Redemption      8358.0           8.5  8.445869
834                      The Godfather      6024.0           8.5  8.425439
10309      Dilwale Dulhania Le Jayenge       661.0           9.1  8.421453
12481                  The Dark Knight     12269.0           8.3  8.265477
2843                        Fight Club      9678.0           8.3  8.256385
292                       Pulp Fiction      8670.0           8.3  8.251406
522                   Schindler's List      4436.0           8.3  8.206639
23673                         Whiplash      4376.0           8.3  8.205404
5481                     Spirited Away      3968.0           8.3  8.196055
2211                 Life Is Beautiful      3643.0           8.3  8.187171
1178            The Godfather: Part II      3418.0           8.3  8.180076
1152   One Flew Over the Cuckoo's Nest      3001.0           8.3  8.164256
351                      

In [25]:
print(c)

6.117420000000009


b

In [29]:
m = df_new['vote_count'].quantile(0.90)

In [30]:
print(m)

381.0


c

In [36]:
print("Movies greater than m: ")
q_movies = df_new.where(df['vote_count']>=m)
print(len(q_movies))

Movies greater than m: 
10000


d IMDb

In [41]:
# Convert columns to numeric safely
df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')

# Mean vote across all movies
C = df['vote_average'].mean()

# 90th percentile of vote counts
m = df['vote_count'].quantile(0.90)

# Qualified movies
q_movies = df.loc[df['vote_count'] >= m].copy()

# Weighted rating formula
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

# Add score column
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

# Top 20 movies sorted by score
top20 = q_movies.sort_values('score', ascending=False).head(20)
print(top20[['title', 'vote_count', 'vote_average', 'score']])


                                 title  vote_count  vote_average     score
314           The Shawshank Redemption      8358.0           8.5  8.445869
834                      The Godfather      6024.0           8.5  8.425439
10309      Dilwale Dulhania Le Jayenge       661.0           9.1  8.421453
12481                  The Dark Knight     12269.0           8.3  8.265477
2843                        Fight Club      9678.0           8.3  8.256385
292                       Pulp Fiction      8670.0           8.3  8.251406
522                   Schindler's List      4436.0           8.3  8.206639
23673                         Whiplash      4376.0           8.3  8.205404
5481                     Spirited Away      3968.0           8.3  8.196055
2211                 Life Is Beautiful      3643.0           8.3  8.187171
1178            The Godfather: Part II      3418.0           8.3  8.180076
1152   One Flew Over the Cuckoo's Nest      3001.0           8.3  8.164256
351                      

TF-IDF

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

# -------------------------------
# 1. Load dataset (first 10,000 rows only)
# -------------------------------
df = pd.read_csv("movies_metadata.csv", low_memory=False)
df_new = df.head(10000).copy()   # proper copy to avoid warnings

# -------------------------------
# 2. Convert vote columns to numeric
# -------------------------------
df_new['vote_average'] = pd.to_numeric(df_new['vote_average'], errors='coerce')
df_new['vote_count'] = pd.to_numeric(df_new['vote_count'], errors='coerce')

# -------------------------------
# 3. Compute C (mean rating) and m (90th percentile of vote counts)
# -------------------------------
C = df_new['vote_average'].mean()
m = df_new['vote_count'].quantile(0.90)

print("Mean vote average (C):", C)
print("90th percentile of vote counts (m):", m)

# -------------------------------
# 4. Filter qualified movies (vote_count >= m)
# -------------------------------
q_movies = df_new.loc[df_new['vote_count'] >= m].copy()
print("Number of qualified movies:", q_movies.shape[0])   # should be ~1001

# -------------------------------
# 5. Weighted rating function
# -------------------------------
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

# Add score column
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

# 6 
top20 = q_movies.sort_values('score', ascending=False).head(20)
print(top20[['title', 'vote_count', 'vote_average', 'score']])

# 7. TF–IDF on overviews (only qualified movies)

# Replace NaN with empty strings
q_movies.loc[:, 'overview'] = q_movies['overview'].fillna('')

# TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Construct TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(q_movies['overview'])

# Shape of matrix
print("TF-IDF shape:", tfidf_matrix.shape)   # ✅ should be (1001, 9013)

# Feature names (indices 5000–5009)
try:
    feature_names = tfidf.get_feature_names_out()
except AttributeError:
    feature_names = tfidf.get_feature_names()

print(feature_names[5000:5010])


Mean vote average (C): 6.117420000000009
90th percentile of vote counts (m): 381.0
Number of qualified movies: 1001
                                              title  vote_count  vote_average  \
314                        The Shawshank Redemption      8358.0           8.5   
834                                   The Godfather      6024.0           8.5   
2843                                     Fight Club      9678.0           8.3   
292                                    Pulp Fiction      8670.0           8.3   
522                                Schindler's List      4436.0           8.3   
5481                                  Spirited Away      3968.0           8.3   
351                                    Forrest Gump      8147.0           8.2   
2211                              Life Is Beautiful      3643.0           8.3   
1178                         The Godfather: Part II      3418.0           8.3   
1154                        The Empire Strikes Back      5998.0           

In [55]:
# -------------------------------
# 8. Show some of the selected movies
# -------------------------------
selected_movies = q_movies['title'].dropna().unique().tolist()

print("Total qualified movies:", len(selected_movies))
print("\nSample of selected movies:")
print(selected_movies[:15])   # first 15 as a preview


Total qualified movies: 998

Sample of selected movies:
['Toy Story', 'Jumanji', 'Heat', 'GoldenEye', 'Balto', 'Casino', 'Four Rooms', 'Ace Ventura: When Nature Calls', 'Assassins', 'Twelve Monkeys', 'Babe', 'Clueless', 'Mortal Kombat', 'Se7en', 'Pocahontas']


In [56]:
# -------------------------------
# 9. Compute Cosine Similarity
# -------------------------------
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print("Cosine similarity matrix shape:", cosine_sim.shape)   # should be (1001, 1001)

# -------------------------------
# 10. Reverse lookup map: movie title -> index
# -------------------------------
q_movies['title_clean'] = q_movies['title'].str.strip().str.lower()

# Build reverse index map
indices = pd.Series(q_movies.index, index=q_movies['title_clean']).drop_duplicates()

# -------------------------------
# 11. Test lookup
# -------------------------------
test_title = "The Dark Knight"
idx = indices.get(test_title.strip().lower(), "Not Found")
print(f"Index of '{test_title}':", idx)


Cosine similarity matrix shape: (1001, 1001)
Index of 'The Dark Knight': Not Found
